In [77]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [78]:
def read_file(file_name):
    file = open(file_name, "r")
    list_string = []
    i = 0
    while True:
        line = file.readline()
        list_string.append(line)
        if not line:
            break
        file.close
    return list_string

In [79]:
file_name = "Sim4ML3.dat"
main_data = []
list_string = read_file(file_name)
# last string is empty
list_string.pop()

''

In [80]:
param_data = []
signal_data = []
for string in list_string:
    data = np.fromstring(string, dtype=float, sep=' ')
    # Removing the row index at the beggining as it's redundant
    data = np.delete(data, 0)
    
    resistance_depth = data[:7]
    param_data.append(resistance_depth)

    signal_values = data[7:57]
    signal_data.append(signal_values)

In [89]:
from sklearn.model_selection import train_test_split

params_train, params_test, signal_train, signal_test = train_test_split(param_data, signal_data, 
                                                    train_size=0.67, 
                                                    random_state=42)


In [90]:
regr = linear_model.LinearRegression()

In [91]:
regr.fit(params_train, signal_train)

LinearRegression()

In [92]:
signal_pred = regr.predict(params_test)
print("Mean squared error: %.25f" % mean_squared_error(signal_test[0], signal_pred[0]))
#print("Coefficients: \n", regr.coef_)
print("Coefficient of determination: %.2f" % r2_score(signal_test, signal_pred))

Mean squared error: 0.0022908280004867000514968
Coefficient of determination: 0.49


In [106]:
signal_train, signal_test, params_train, params_test = train_test_split(param_data, signal_data, 
                                                    train_size=0.67, 
                                                    random_state=42)

In [107]:
regr = linear_model.LinearRegression()

In [108]:
regr.fit(signal_train, params_train)

LinearRegression()

In [109]:
params_pred = regr.predict(signal_test)
print("Mean squared error: %.25f" % mean_squared_error(params_test, params_pred))
#print("Coefficients: \n", regr.coef_)
print("Coefficient of determination: %.2f" % r2_score(params_test, params_pred))

Mean squared error: 0.0039731986443257870031398
Coefficient of determination: 0.49
